In [1]:
import os
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import json
import numpy as np
import timeit 
from datetime import datetime

In [12]:
# Cufflinks wrapper on plotly
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot
cufflinks.go_offline()

# Set global theme

import plotly.figure_factory as ff

import plotly.graph_objects as go

In [3]:
df_params=pd.read_excel("params.xlsx")
df_location=pd.read_excel("location.xlsx")

In [5]:
# Basic data viz

df_params.columns

Index(['Unnamed: 0', 'Area', 'AreaRange', 'Area_from', 'Area_to', 'Build_year',
       'Building_floors_num', 'Building_material', 'Building_ownership',
       'Building_type', 'City', 'City_id', 'Construction_status', 'Country',
       'Equipment_types', 'Extras_types', 'Floor_no', 'Floors_num', 'Heating',
       'Id', 'MarketType', 'Media_types', 'OfferType', 'Offered_estates_type',
       'Photo', 'Price', 'PriceRange', 'Price_from', 'Price_per_m',
       'Price_per_m_from', 'Price_to', 'ProperType', 'Province', 'RegularUser',
       'Rent', 'Rooms_num', 'Security_types', 'State', 'Subregion', 'Title',
       'Windows_type', 'env', 'seller_id', 'user_type'],
      dtype='object')

In [6]:
df_params.shape

(2684, 44)

In [7]:
df_location.head()

,Unnamed: 0,Unnamed: 1,Id,address,city,district,latitude,longitude,radius,region,sub-region,zoom_level
0,0,NaN,60495264,"Warszawa, Białołęka, Żerań, Płochocińska",Warszawa,Żerań,52.337440,21.029168,0,mazowieckie,Warszawa,15
1,1,NaN,60495358,"Warszawa, Praga-Południe, Grochów, ul. Chrzano...",Warszawa,Grochów,52.253734,21.082111,0,mazowieckie,Warszawa,14
2,2,NaN,60054182,"Warszawa, Mokotów, ul. Cybernetyki",Warszawa,Mokotów,52.176766,20.989376,0,mazowieckie,Warszawa,15
3,3,NaN,60495197,"Warszawa, Praga-Północ, Ireny Kosmowskiej",Warszawa,Praga-Północ,52.259860,21.050550,0,mazowieckie,Warszawa,15
4,4,NaN,60305076,"Warszawa, Bemowo",Warszawa,Bemowo,52.243815,20.912353,100,mazowieckie,Warszawa,16


In [8]:
df_pricing=df_params[["Id","Area","Price",'Price_per_m']]

In [9]:
df=pd.merge(df_pricing,df_location[["Id","latitude","longitude","city","district"]],left_on="Id",right_on="Id")

In [57]:
df=df.query("city=='Warszawa'")
df=df.query("latitude<54")

In [131]:
df.groupby("district", as_index=False).count()[["district","Id"]].sort_values(by="Id",ascending=False).head(10)

,district,Id
86,Wola,285
35,Mokotów,271
4,Białołęka,198
94,Śródmieście,197
3,Bemowo,161
48,Praga-Południe,139
79,Ursynów,106
84,Wilanów,94
78,Ursus,80
6,Bielany,73


In [119]:
data=[]
trace0=dict(
            lat=df.latitude,
            lon=df.longitude,
          
            type="scattermapbox",
            mode='markers',
            marker=dict(
                opacity=0.5, 
                size=10, 
                color=df.Price_per_m,
               
                colorscale="jet",
                cauto=False,
                cmax=20000,
                showscale=True),
                text=df.Price_per_m,
            
           
            )



data=[trace0]

layout = dict(
    
   
    height = 800,
    # top, bottom, left and right margins
    margin = dict(t = 0, b = 0, l = 0, r = 0),
    font = dict(color = '#FFFFFF', size = 11),
    paper_bgcolor = '#000000',
    mapbox = dict(
        # here you need the token from Mapbox
        accesstoken ="pk.eyJ1IjoiamFubWFqZXdza2kiLCJhIjoiY2p1bnRkN2JiMTFuczRjbXFuaDg0YWswZSJ9.LICCkkSfzPTXlNZ3MC-HHg",
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 52.2,
            lon = 21
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 10,
        # default map style
        style = 'light'
    )
)

figure=dict(
    data=data,
    layout=layout
    )
iplot(figure)

In [120]:
df.describe()

,Id,Area,Price,Price_per_m,latitude,longitude,lon_mod,lat_mod
count,2.611000e+03,2554.000000,2.554000e+03,2496.000000,2611.000000,2611.000000,2611.000000,2611.000000
mean,6.024897e+07,65.304675,6.949405e+05,11380.388622,52.230281,21.005122,21.005167,52.230057
std,1.137749e+06,199.298746,5.872536e+05,3782.983355,0.048344,0.064922,0.065014,0.048226
min,3.868593e+07,15.850000,0.000000e+00,94.000000,52.098421,20.856410,20.860000,52.100000
25%,6.036031e+07,42.000000,4.273505e+05,9246.500000,52.194700,20.956340,20.960000,52.190000
50%,6.047993e+07,54.945000,5.550000e+05,10781.500000,52.232043,21.006020,21.010000,52.230000
75%,6.048883e+07,70.000000,7.622500e+05,12670.000000,52.252664,21.049372,21.050000,52.250000
max,6.049536e+07,10000.000000,7.650000e+06,48900.000000,52.360711,21.246374,21.250000,52.360000


In [146]:
(df.lon_mod-df.longitude).describe()

count    2611.000000
mean        0.000011
std         0.029294
min        -0.050000
25%        -0.024911
50%         0.000310
75%         0.023546
max         0.049999
dtype: float64

In [173]:
df["lon_mod"]=np.round(df.longitude,2)
#df["lon_mod"]=np.where((df.lon_mod-df.longitude)<0,df.lon_mod+0.05,df.lon_mod)

df["lat_mod"]=np.round(df.latitude,2)
#df["lat_mod"]=np.where((df.lat_mod-df.latitude)<0,df.lat_mod+0.05,df.lat_mod)

In [174]:
df_piv=df.pivot_table(index="lat_mod",columns="lon_mod",values="Price_per_m",aggfunc=np.mean)

In [175]:
arr=df_piv.values

In [176]:
df_piv.shape

(27, 38)

In [177]:
ones=np.ones(df_piv.shape)

In [178]:
df_piv.values

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [179]:
df_piv.columns.shape

(38,)

In [180]:
df_piv.index.shape

(27,)

In [181]:
fig = go.Figure(data=go.Heatmap(
                   z=df_piv.values,
                   y=df_piv.index,
                   x=df_piv.columns,
                 ))
fig.show()

In [182]:
trace0=go.Surface(
        z=df_piv.values,
        y=df_piv.index,
        x=df_piv.columns,
       connectgaps=True
          

        )

data=[trace0]

In [183]:
figure=go.Figure(
    data=data,
    layout=go.Layout(
      
        scene=dict(
            xaxis=dict(title="Longitude"),
            yaxis=dict(title="Latitude"),
            zaxis=dict(title="Price per m2")

           
                  ),
         title="Real estate price per m2 geo distribution"
    ))
iplot(figure)

In [184]:
fig = go.Figure(data =
    go.Contour(
        z=df_piv.values,
        y=df_piv.index,
        x=df_piv.columns,
        connectgaps=True
    ))
fig.show()